This notebook implements a Gurobi model to solve the assignment problem. The following code block imports libraries that are used in the implementation.

In [1]:
import itertools
import json

import gurobipy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

#### Formulation

<u>Set(s)</u></br>
$M$ - set of resources, $i \in M$</br>
$N$ - set of tasks, $j \in N$</br>

<u>Parameters</u></br>
$c_{ij}$ - cost of assigning resource $i$ to task $j$

<u>Decision Variables</u></br>
$X_{ij}$ - 1, if resource $i$ is assigned to task $j$; 0, otherwise.

\begin{align}
\text{Minimize } \sum_{i\in M}\sum_{j\in N} c_{ij}X_{ij} \hspace{10cm}~\\
\end{align}
\begin{align}
\text{subject }to\qquad\qquad\\
\sum_{i \in M} X_{ij} &= 1, \forall j \in N. && \text{(Each task assigned to one resource)}\\
\sum_{j \in N} X_{ij} &= 1, \forall i \in M. && \text{(Each resource assigned to one task)}\\
X_{ij} & \in \{0, 1\}, \forall i \in M, j \in N. && \text{(Assignment variables are binary)}
\end{align}

The following code block generates data for an example instance of the assignment problem

In [2]:
SIZE = 10

M = {i for i in range(1, SIZE+1)}
N = {j for j in range(1, SIZE+1)}

np.random.seed(0)

c = {(i, j): np.random.uniform(low=0.0, high=10.0) for i, j in itertools.product(M, N)}

First, we need to instantiate a Gurobi `Model` instance

In [3]:
model = gurobipy.Model('Assignment-Problem')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-30


Next, we define the decision variables

In [4]:
X = model.addVars(
    N, 
    M, 
    vtype=gurobipy.GRB.BINARY,
    name='X',
)

Next, we define the objective function and specify that our instantiated model should use the defined objective. As a reminder, our objective is:

$$\text{Minimize } \sum_{i\in M}\sum_{j\in N} c_{ij}X_{ij}.$$

Note that we use a `LinExpr` object and `for` loops to capture the sums in the expression.

In [5]:
total_cost = gurobipy.LinExpr()
for i in N:
    for j in M:
        total_cost.add(c[i, j]*X[i, j])

model.setObjective(
    total_cost, 
    sense=gurobipy.GRB.MINIMIZE,
)

The following code block defines the set of constraints that ensure each task is assigned to one resource. In the formulation, we represented this set of constraints as:

$$\sum_{i \in M} X_{ij} = 1, \forall j \in N.$$

In [6]:
for j in N:
    i_sum = gurobipy.LinExpr()
    for i in M:
        i_sum.add(X[i, j])
    model.addConstr(
        i_sum == 1, 
        name=f'task_{j}_assigned_to_one_resource',
    )

The following code block defines the set of constraints that ensure each resource is assigned to one task. In the formulation, we represented this set of constraints as:

$$\sum_{j \in N} X_{ij} = 1, \forall i \in M.$$

In [7]:
for i in M:
    j_sum = gurobipy.LinExpr()
    for j in N:
        j_sum.add(X[i, j])
    model.addConstr(
        j_sum == 1, 
        name=f'resource_{i}_assigned_to_one_task',
    )

Before we solve the model, lets `update` our model and look at the information it contains.

In [8]:
model.update()
model

<gurobi.Model MIP instance Assignment-Problem: 20 constrs, 100 vars, Parameter changes: Username=(user-defined)>

Let's look at the constraints

In [9]:
model.getConstrs()

[<gurobi.Constr task_1_assigned_to_one_resource>,
 <gurobi.Constr task_2_assigned_to_one_resource>,
 <gurobi.Constr task_3_assigned_to_one_resource>,
 <gurobi.Constr task_4_assigned_to_one_resource>,
 <gurobi.Constr task_5_assigned_to_one_resource>,
 <gurobi.Constr task_6_assigned_to_one_resource>,
 <gurobi.Constr task_7_assigned_to_one_resource>,
 <gurobi.Constr task_8_assigned_to_one_resource>,
 <gurobi.Constr task_9_assigned_to_one_resource>,
 <gurobi.Constr task_10_assigned_to_one_resource>,
 <gurobi.Constr resource_1_assigned_to_one_task>,
 <gurobi.Constr resource_2_assigned_to_one_task>,
 <gurobi.Constr resource_3_assigned_to_one_task>,
 <gurobi.Constr resource_4_assigned_to_one_task>,
 <gurobi.Constr resource_5_assigned_to_one_task>,
 <gurobi.Constr resource_6_assigned_to_one_task>,
 <gurobi.Constr resource_7_assigned_to_one_task>,
 <gurobi.Constr resource_8_assigned_to_one_task>,
 <gurobi.Constr resource_9_assigned_to_one_task>,
 <gurobi.Constr resource_10_assigned_to_one_task>

Let's look at the objective

In [10]:
model.getObjective()

<gurobi.LinExpr: 5.4881350392732475 X[1,1] + 7.151893663724195 X[1,2] + 6.027633760716439 X[1,3] + 5.448831829968968 X[1,4] + 4.236547993389047 X[1,5] + 6.458941130666561 X[1,6] + 4.375872112626925 X[1,7] + 8.917730007820797 X[1,8] + 9.636627605010293 X[1,9] + 3.8344151882577773 X[1,10] + 7.917250380826646 X[2,1] + 5.288949197529044 X[2,2] + 5.680445610939323 X[2,3] + 9.25596638292661 X[2,4] + 0.7103605819788694 X[2,5] + 0.8712929970154071 X[2,6] + 0.2021839744032572 X[2,7] + 8.32619845547938 X[2,8] + 7.781567509498505 X[2,9] + 8.700121482468191 X[2,10] + 9.78618342232764 X[3,1] + 7.9915856421672355 X[3,2] + 4.6147936225293185 X[3,3] + 7.805291762864554 X[3,4] + 1.1827442586893322 X[3,5] + 6.399210213275238 X[3,6] + 1.433532874090464 X[3,7] + 9.446689170495839 X[3,8] + 5.218483217500717 X[3,9] + 4.146619399905235 X[3,10] + 2.64555612104627 X[4,1] + 7.742336894342166 X[4,2] + 4.5615033221654855 X[4,3] + 5.684339488686485 X[4,4] + 0.18789800436355142 X[4,5] + 6.176354970758771 X[4,6] + 6

The following code block solves the defined optimization model

In [11]:
model.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Pop!_OS 22.04 LTS")

CPU model: AMD Ryzen 7 5800X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 20 rows, 100 columns and 200 nonzeros
Model fingerprint: 0x2395cee8
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-02, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 55.5823369
Presolve time: 0.00s
Presolved: 20 rows, 100 columns, 200 nonzeros
Variable types: 0 continuous, 100 integer (100 binary)

Root relaxation: objective 1.437781e+01, 18 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      14.3778109  

When the model is solved, you can use the `getJSONSolution` method to get information regarding the solution.

In [12]:
json.loads(model.getJSONSolution())

{'SolutionInfo': {'Status': 2,
  'Runtime': 0.008269071578979492,
  'Work': 0.0005920507815653001,
  'ObjVal': 14.377810918667794,
  'ObjBound': 14.377810918667793,
  'ObjBoundC': 14.377810918667793,
  'MIPGap': 0,
  'IntVio': 0,
  'BoundVio': 0,
  'ConstrVio': 0,
  'IterCount': 18,
  'BarIterCount': 0,
  'NodeCount': 1,
  'SolCount': 2,
  'PoolObjBound': 14.377810918667793,
  'PoolObjVal': [14.377810918667794, 55.58233689954017]},
 'Vars': [{'VarName': 'X[1,10]', 'X': 1},
  {'VarName': 'X[2,7]', 'X': 1},
  {'VarName': 'X[3,5]', 'X': 1},
  {'VarName': 'X[4,1]', 'X': 1},
  {'VarName': 'X[5,9]', 'X': 1},
  {'VarName': 'X[6,4]', 'X': 1},
  {'VarName': 'X[7,2]', 'X': 1},
  {'VarName': 'X[8,6]', 'X': 1},
  {'VarName': 'X[9,3]', 'X': 1},
  {'VarName': 'X[10,8]', 'X': 1}]}

You can get the value of the objective by calling the `ObjVal` attribute 

In [13]:
model.ObjVal

14.377810918667794

If you want to access the specific value of a decision variable, you can use the variables `x` attribute

In [14]:
X[1, 1].x

0.0

The following code block prints the optimal assignments for the sample problem

In [15]:
print('Optimal Assignments')
print('-'*20)
for i in M:
    for j in N:
        if X[i, j].x > 0.1:
            print(f' - Resource {i} -> Task {j}')
            break

Optimal Assignments
--------------------
 - Resource 1 -> Task 10
 - Resource 2 -> Task 7
 - Resource 3 -> Task 5
 - Resource 4 -> Task 1
 - Resource 5 -> Task 9
 - Resource 6 -> Task 4
 - Resource 7 -> Task 2
 - Resource 8 -> Task 6
 - Resource 9 -> Task 3
 - Resource 10 -> Task 8


#### All together

The following code block includes all of the code to generate data and solve the model.

In [16]:
SIZE = 10

M = {i for i in range(1, SIZE+1)}
N = {j for j in range(1, SIZE+1)}

np.random.seed(0)

c = {(i, j): np.random.uniform(low=0.0, high=10.0) for i, j in itertools.product(M, N)}

model = gurobipy.Model('Assignment-Problem')

X = model.addVars(
    N, 
    M, 
    vtype=gurobipy.GRB.BINARY,
    name='X',
)

total_cost = gurobipy.LinExpr()
for i in N:
    for j in M:
        total_cost.add(c[i, j]*X[i, j])

model.setObjective(
    total_cost, 
    sense=gurobipy.GRB.MINIMIZE,
)

for j in N:
    i_sum = gurobipy.LinExpr()
    for i in M:
        i_sum.add(X[i, j])
    model.addConstr(
        i_sum == 1, 
        name=f'task_{j}_assigned_to_one_resource',
    )

for i in M:
    j_sum = gurobipy.LinExpr()
    for j in N:
        j_sum.add(X[i, j])
    model.addConstr(
        j_sum == 1, 
        name=f'resource_{i}_assigned_to_one_task',
    )

model.optimize()

print('Optimal Assignments')
print('-'*20)
for i in M:
    for j in N:
        if X[i, j].x > 0.1:
            print(f' - Resource {i} -> Task {j}')
            break

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Pop!_OS 22.04 LTS")

CPU model: AMD Ryzen 7 5800X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 20 rows, 100 columns and 200 nonzeros
Model fingerprint: 0x2395cee8
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-02, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 55.5823369
Presolve time: 0.00s
Presolved: 20 rows, 100 columns, 200 nonzeros
Variable types: 0 continuous, 100 integer (100 binary)

Root relaxation: objective 1.437781e+01, 18 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      14.3778109  